In [1]:
import pymysql
from config.DatabaseConfig import * # DB 접속 정보 불러오기

db = None
try:
    db = pymysql.connect(
        host=DB_HOST,
        user=DB_USER,
        passwd=DB_PASSWORD,
        db=DB_NAME,
        charset='utf8'
    )

    # 테이블 생성 sql 정의
    sql = '''
      CREATE TABLE IF NOT EXISTS `chatbot_train_data` (
      `id` INT UNSIGNED NOT NULL AUTO_INCREMENT,
      `intent` VARCHAR(45) NULL,
      `ner` VARCHAR(1024) NULL,
      `query` TEXT NULL,
      `answer` TEXT NOT NULL,
      `answer_image` VARCHAR(2048) NULL,
      PRIMARY KEY (`id`))
    ENGINE = InnoDB DEFAULT CHARSET=utf8
    '''

    # 테이블 생성
    with db.cursor() as cursor:
        cursor.execute(sql)
        
    print('테이블 생성 성공')

except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()



테이블 생성 성공


In [2]:
import pandas as pd
df = pd.read_excel(r'C:/DevRoot/TeamProj2/train_data/train_data_q&a.xlsx')
df

,의도,개체명,질문,답변,답변이미지
0,상품주문,NaN,주문할래,총 몇 개 구매하시겠어요?,NaN
1,할인적용문의,NaN,할인 받을 수 있어?,고객님이 보유하신 쿠폰 사용 시 할인받으실 수 있어요.,NaN
2,상품색상문의,NaN,무슨색이야?,색상은 다음과 같아요,NaN
3,반품요청,NaN,저번에 주문한 것 반품하고싶어,해당 주문에 대해 반품 원하시는 것 맞으신가요?,NaN
4,상품가격문의,NaN,얼마야?,이 상품의 가격은 다음과 같습니다.,NaN
5,주문취소요청,NaN,주문 취소할래,해당 주문에 대해 취소 원하시는 것 맞으신가요?,NaN
6,상품사이즈문의,NaN,사이즈가 어떻게돼?,사이즈는 다음과 같아요.,NaN
7,배송일정확인,NaN,주문한 물건 언제와?,주문하신 상품이 아직 출고되지 않았습니다.,NaN
8,상품추천요청,{B_CATEGORY},{B_CATEGORY} 제품 추천해줘,"고객님의 취향에 맞춰 골라봤어요. ,",NaN
9,상품추천요청,NaN,제품 추천해줘,어떤 종류의 상품을 추천해 드릴까요?,NaN


In [3]:
import pymysql
import openpyxl

from config.DatabaseConfig import * # DB 접속 정보 불러오기


In [4]:
# 학습 데이터 초기화
def all_clear_train_data(db):
    # 기존 학습 데이터 삭제
    sql = '''
            delete from chatbot_train_data
        '''
    with db.cursor() as cursor:
        cursor.execute(sql)

    # auto increment 초기화
    sql = '''
    ALTER TABLE chatbot_train_data AUTO_INCREMENT=1
    '''
    with db.cursor() as cursor:
        cursor.execute(sql)


# db에 데이터 저장
def insert_data(db, xls_row):
    intent, ner, query, answer, answer_img_url = xls_row

    sql = '''
        INSERT chatbot_train_data(intent, ner, query, answer, answer_image) 
        values(
         '%s', '%s', '%s', '%s', '%s'
        )
    ''' % (intent.value, ner.value, query.value, answer.value, answer_img_url.value)

    # 엑셀에서 불러온 cell에 데이터가 없는 경우, null 로 치환
    sql = sql.replace("'None'", "null")

    with db.cursor() as cursor:
        cursor.execute(sql)
        print('{} 저장'.format(query.value))
        db.commit()


In [5]:
train_file = r'C:/DevRoot/TeamProj2/train_data/train_data_q&a.xlsx'
db = None
try:
    db = pymysql.connect(
        host=DB_HOST,
        user=DB_USER,
        passwd=DB_PASSWORD,
        db=DB_NAME,
        charset='utf8'
    )

    # 기존 학습 데이터 초기화
    all_clear_train_data(db)

    # 학습 엑셀 파일 불러오기
    wb = openpyxl.load_workbook(train_file)
    sheet = wb['Sheet1']
    for row in sheet.iter_rows(min_row=2): # 해더는 불러오지 않음
        # 데이터 저장
        insert_data(db, row)

    wb.close()

except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()



주문할래 저장
할인 받을 수 있어? 저장
무슨색이야? 저장
저번에 주문한 것 반품하고싶어 저장
얼마야? 저장
주문 취소할래 저장
사이즈가 어떻게돼? 저장
주문한 물건 언제와? 저장
{B_CATEGORY} 제품 추천해줘 저장
제품 추천해줘 저장
뭐시기개 저장
{B_CATEGORY} 저장
예 저장
아니오 저장


In [6]:
def _make_query(intent_name, ner_tags):
    sql = "select * from chatbot_train_data"
    if intent_name != None and ner_tags == None:
        sql = sql + " where intent='{}' ".format(intent_name)

    elif intent_name != None and ner_tags != None:
        where = ' where intent="%s" ' % intent_name
        if (len(ner_tags) > 0):
            where += 'and ('
            for ne in ner_tags:
                where += " ner like '%{}%' or ".format(ne)
            where = where[:-3] + ')'
        sql = sql + where 
        
    # 동일한 답변이 2개 이상인 경우 랜덤으로 선택
    sql = sql + " order by rand() limit 1"
    return sql;

In [7]:
print(_make_query('추천', ['B_CATEGORY']))

select * from chatbot_train_data where intent="추천" and ( ner like '%B_CATEGORY%' ) order by rand() limit 1


In [1]:
from config.DatabaseConfig import *
from utils.Database import Database
from utils.Preprocess import Preprocess
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing


In [2]:
# 전처리 객체 생성
p = Preprocess(word2index_dic='./train_tools/dict/chatbot_dict.bin',
               userdic='./utils/user_dic.tsv')


In [3]:
# 질문/답변 학습 디비 연결 객체 생성
db = Database(
    host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME
)
db.connect()    # 디비 연결

In [4]:
# 질의 문장
query = "바지 추천해주세요"

In [5]:
# 의도 분류
from models.intent.IntentModel import IntentModel
intent = IntentModel(model_name='./models/intent/intent_model.h5', preprocess=p)

In [6]:
predict = intent.predict_class(query)
predict

8

In [7]:
intent_name = intent.labels[predict]
intent_name

'상품추천요청'

In [8]:
# 개체명 인식 (NER)
from models.ner.NerModel import NerModel
ner = NerModel(model_name='./models/ner/ner_model.h5', preprocess=p)

In [9]:
predicts = ner.predict(query)
predicts

[('바지', 'B_CATEGORY'), ('추천', 'I'), ('주', 'NNP')]

In [10]:
ner_tags = ner.predict_tags(query)
ner_tags

['B_CATEGORY', 'I', 'NNP']

In [11]:
print("질문 : ", query)
print("=" * 100)
print("의도 파악 : ", intent_name)
print("개체명 인식 : ", predicts)
print("답변 검색에 필요한 NER 태그 : ", ner_tags)
print("=" * 100)

질문 :  바지 추천해주세요
의도 파악 :  상품추천요청
개체명 인식 :  [('바지', 'B_CATEGORY'), ('추천', 'I'), ('주', 'NNP')]
답변 검색에 필요한 NER 태그 :  ['B_CATEGORY', 'I', 'NNP']


In [12]:
# 답변검색
from utils.FindAnswer import FindAnswer

f = FindAnswer(db)
answer_text, answer_image = f.search(intent_name, ner_tags)
answer = f.tag_to_word(predicts, answer_text)
try:
    f = FindAnswer(db)
    answer_text, answer_image = f.search(intent_name, ner_tags)
    answer = f.tag_to_word(predicts, answer_text)
except:
    answer = "죄송해요 무슨 말인지 모르겠어요"

print("답변 : ", answer)

답변 :  고객님의 취향에 맞춰 골라봤어요. ,22SS 스트레치 유틸리티 카고 조거 팬츠_블랙


In [13]:
answer

'바지'